<a href="https://colab.research.google.com/github/GreihMurray/NLP-2/blob/AG_Murray/irish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional
from tensorflow.keras.optimizers import SGD
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
import joblib
import pickle
import nltk

In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
def read_file_to_sents():
    with open("train.tsv", encoding="utf-8") as file:
        f = csv.reader(file, delimiter="\t")

        cur_sent = []
        all_sents = []

        for line in tqdm(f, desc="Reading data..."):
            if line[0][0:2].strip() == 'N':
                line[0] = 'N'

            if line[0] == "<S>":
                if len(cur_sent) >= 1:
                    all_sents.append(cur_sent)
                cur_sent = []
                continue

            cur_sent.append((line[0], line[1]))

    return all_sents

In [5]:
def string_to_num(data):
    le = LabelEncoder()

    label = le.fit_transform(data)

    return label

In [6]:
def defeature(data):
    unfeatured = []

    for dict in data:
        unfeatured.append(dict['word'])

    return unfeatured

In [7]:
def seq_model(x, y):
    model = Sequential()
    model.add(Dense(256, input_shape=(1,), activation="sigmoid"))
    model.add(Dense(128, activation="sigmoid"))
    model.add(Dense(10, activation="softmax"))
    model.add(Dense(1, activation="softmax"))

  # From https://towardsdatascience.com/hyperparameter-tuning-with-kerastuner-and-tensorflow-c4a4d690b31a
    #stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

    x = defeature(x)

    x = string_to_num(x)
    y = string_to_num(y)

    x = np.asarray(x)
    y = np.asarray(y)

    print("[INFO] training network...")
    sgd = SGD(0.01)
    model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])
    H = model.fit(x, y, epochs=100, batch_size=100) #, callbacks=[stop_early])

    return model

In [8]:
def eval_model(model, x_test, y_test):
  x_test = defeature(x_test)
  x_test = string_to_num(x_test)
  y_test = string_to_num(y_test)
  return model.evaluate(x_test[:5000], y_test[:5000], return_dict=True)

Below work partially https://nlpforhackers.io/training-pos-tagger/amp/

In [9]:
def dec_tree(X, y):
    clf = Pipeline([
        ('vectorizer', DictVectorizer(sparse=False)),
        ('classifier', DecisionTreeClassifier(criterion='entropy')),
    ], verbose=1)

    print("Training Started")

    # Custom work below
    all_clfs = []

    for i in tqdm(range(0, 115), desc="Training"):
        cur_clf = clf
        cur_clf.fit(X[(i * 31649): ((i + 1) * 31649)], y[(i * 31649): ((i + 1) * 31649)])  # Use only the first 10K samples if you're running it multiple times. It takes a fair bit :)

        all_clfs.append(cur_clf)

        file_loc = '/content/gdrive/MyDrive/Colab_Notebooks/NLP/model' + str(i) + ".h5"
        file_loc_2 = '/gdrive/MyDrive/Colab_Notebooks/NLP/vector' + str(i) + ".pkl"

        # cur_clf.named_steps['classifier'].save(file_loc)

        with open(file_loc, 'wb') as pickle_file:
          pickle.dump(cur_clf, pickle_file)

        # cur_clf.named_steps['estimator'].model = None

        # joblib.dump(cur_clf, file_loc_2)

    print('Training completed')

    return all_clfs

In [10]:
def acc_score(all_clfs, x_test, y_test):
    all_scores = []

    for clf in tqdm(all_clfs, desc="evaluating..."):
      temp_scores = []
      for i in range(0,50):
        temp_scores.append(clf.score(x_test[int(i * (len(x_test) / 250)):int( (i + 1) * (len(x_test)/250))], y_test[int(i * (len(x_test) / 250)):int( (i + 1) * (len(x_test)/250))]))

        all_scores.append(sum(temp_scores)/ len(temp_scores))

    total_acc = 0

    for score in all_scores:
        total_acc += score

    print("Accuracy:", total_acc / len(all_scores))

In [11]:
# Below work comes from https://nlpforhackers.io/training-pos-tagger/amp/

In [12]:
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }

In [13]:
# Below work comes from https://nlpforhackers.io/training-pos-tagger/amp/

In [14]:
def untag(tagged_sentence):
    return [w for w, t in tagged_sentence]

In [15]:
# Below work comes from https://nlpforhackers.io/training-pos-tagger/amp/

In [16]:
def transform_to_dataset(tagged_sentences):
    X, y = [], []

    for tagged in tagged_sentences:
        for index in range(len(tagged)):
            X.append(features(untag(tagged), index))
            y.append(tagged[index][1])

    return X, y

In [17]:
def load_models():
  all_clfs = []

  for i in range(1, 115):
    file_loc = '/content/gdrive/MyDrive/Colab_Notebooks/NLP/model' + str(i) + ".h5"

        # cur_clf.named_steps['classifier'].save(file_loc)

    with open(file_loc, 'rb') as pickle_file:
      cur_clf = pickle.load(pickle_file)

      all_clfs.append(cur_clf)

  return all_clfs



Below based on code provided by Dr. Scannell

In [18]:
def HMM_uni(train, test):
    uni_tag = nltk.UnigramTagger(train)
    acc = uni_tag.accuracy(test)

    print("Unigram tagger accuracy: ", acc)

In [19]:
def gen_rule_tag():
    patterns = [
        (r'a .*', 'H'),
        (r'an .*', 'H'),
        (r'mo .*', 'H'),
        (r'do .*', 'H')
    ]

    rule_tagger = nltk.RegexpTagger(patterns)

    return rule_tagger

Below based on code provided by Dr. Scannell

In [38]:
def HMM_bi(train, test):
    rule_tagger = gen_rule_tag()

    print(train[:5])
    print(len(train[0]))

    bigrams = gen_grams(train, 1)
    
    print(bigrams[0:5])
    print(len(bigrams[0]))

    bitag = nltk.UnigramTagger(bigrams, backoff=rule_tagger)
    acc = bitag.accuracy(test)

    print("Bigram tagger accuracy: ", acc)

In [21]:
def HMM_tri(train, test):
    tritag = nltk.TrigramTagger(train)

    acc = tritag.accuracy(test)

    print("Trigram tagger accuracy: ", acc)

In [22]:
def sent_to_list(data):
    all_data = []

    for entry in data:
      for tup in entry:
        all_data.append(tup)

    return all_data

In [44]:
def gen_grams(data, n = 1):
    all_grams = []
    
    for sent in data:
      cur_grams = [(' '.join([sent[i][0], sent[i+1][0]]), sent[i+1][1]) for i in range(len(sent)-1)]

      cur_grams.append(sent[len(sent)-1])

      all_grams.append(cur_grams)

    return all_grams

In [24]:
def anything():
    data = read_file_to_sents()

    print('Splitting data')

  # from https://nlpforhackers.io/training-pos-tagger/amp/
    cutoff = int(.75 * len(data))
    training_sentences = data[:cutoff]
    test_sentences = data[cutoff:]

    print('Transforming training data')

    X, y = transform_to_dataset(training_sentences)

    del data

    print('Transforming test data')

    x_test, y_test = transform_to_dataset(test_sentences)

    print('Loading models')

  # Original
    #all_clfs = dec_tree(X, y) # Training Decision Tree Models
    #all_clfs = load_models() # Loading models
    #acc_score(all_clfs, x_test, y_test) # Calculating accuracy

    # model = seq_model(X, y) # Train sequential model
    # print(eval_model(model, x_test, y_test)) # Eval sequential model

    # model.save('/content/gdrive/MyDrive/Colab_Notebooks/NLP/seq_model.h5')

    HMM_uni(training_sentences, test_sentences)

    HMM_bi(training_sentences, test_sentences)

    HMM_tri(training_sentences, test_sentences)

    del training_sentences, test_sentences

In [45]:
anything()

Reading data...: 5057059it [00:26, 188745.68it/s]


Splitting data
Transforming training data
Transforming test data
Loading models
Unigram tagger accuracy:  0.8907771596737659
[[('ansin', 'N'), (')', 'N'), ('tá', 'N'), ('níos', 'N'), ('lú', 'N'), ('gaeilge', 'N'), ('ag', 'N'), ('na', 'N'), ('gardaí', 'N'), ('ná', 'N'), ('bí', 'S'), ('ariamh', 'N'), ('ainneoin', 'N'), ('na', 'N'), ('cearta', 'U'), ('.', 'N'), ('níl', 'N'), ('sé', 'N'), ('ach', 'N'), ('roinnt', 'N'), ('seachtainí', 'N'), ('ó', 'N'), ('sin', 'S'), ('a', 'N'), ('tógadh', 'N'), ('fear', 'N'), ('bocht', 'N'), ('a', 'N'), ('tug', 'S'), ('ainm', 'N'), ('gaeilge', 'N'), ('dóibh', 'N'), ('.', 'N')], [('socraíodh', 'N'), ('go', 'N'), ('raibh', 'N'), ('gá', 'N'), ('lena', 'N'), ('leithéid', 'N'), (',', 'N'), ('mar', 'N'), ('go', 'N'), ('bíonn', 'U'), ('na', 'N')], [('tá', 'N'), ('an', 'N'), ('córas', 'N'), ('bainistíochta', 'N'), ('tar', 'N'), ('éis', 'N'), ('freastal', 'N'), ('go', 'N'), ('maith', 'N'), ('ar', 'N'), ('rialtas', 'N'), ('áitiúil', 'N'), ('na', 'N'), ('éireann', 'H'

KeyboardInterrupt: ignored